In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

In [2]:
orgs_column = pd.read_csv("data_files/fomprof-audit-20220318-Fund-Org-ADV - Orgs.csv")
funds_column = pd.read_csv("data_files/fomprof-audit-20220318-Fund-Org-ADV - Funds.csv")

In [13]:
#The Funds file had some random cells with "(blank), " in them that need to be removed
#funds_column.drop(funds_column.index[funds_column['Row Labels'] == '(blank)'], inplace = True)
"""
This caused errors in the next block
I deleted the four cells that said '(blank)' by hand
"""

"\nThis caused errors in the next block\nI deleted the four cells that said '(blank)' by hand\n"

In [8]:
def make_series(in_person_list, num_columns):
    
    temp_series = pd.Series({
        'Username' : in_person_list[0],
        'Full Name': in_person_list[1]
    })

    count = 1
    for element in in_person_list[2:]:
        col_name = "Num " + str(count)
        temp_series = \
            temp_series.append(pd.Series({
                col_name : element
            }))
        count +=1

    for x in range(count, num_columns):
        col_name = "Num " + str(x)
        temp_series = \
            temp_series.append(pd.Series({
                col_name : pd.NA
            }))
        count +=1
        
    return temp_series
        

def break_column(in_column_df):
    list_of_lists = []
    temp_list = []
    first_of_two = True
    count = 0
    max_elements = 0
    
    for index, row in in_column_df.iterrows():
        current_element = in_column_df.iloc[index, 0]
        try: #its an account number
            temp_list.append(int(current_element))
            count += 1

        except: #We've reached a new individual
            if first_of_two:
                list_of_lists.append(temp_list.copy())
                temp_list = [current_element]
                first_of_two = not first_of_two
                if count > max_elements:
                    max_elements = count
                count = 0
            else:
                temp_list.append(current_element)
                first_of_two = not first_of_two
    
    list_of_lists.pop(0) 
    #Because of the boolean logic abvoe, the first list is always empty
    
    
    list_of_lists = Parallel(n_jobs = -1, batch_size = 5)(
        delayed(make_series)(person_list, max_elements) for person_list in list_of_lists)
    #Probably overkill, it was already pretty fast in series
    
    return pd.DataFrame(list_of_lists)

In [9]:
orgs_df = break_column(orgs_column)
funds_df = break_column(funds_column)

In [10]:
staff_titles = pd.read_excel("data_files/Employees Security Rights 2 Funds.xlsx")

In [11]:
orgs_df = pd.merge(staff_titles, orgs_df, left_on = "Banner Finance User ID", right_on = "Username")\
    .drop(['Full Name', 'Emp ID', 'Position Number', 'Banner Finance User ID'], axis = 1)
funds_df = pd.merge(staff_titles, funds_df, left_on = "Banner Finance User ID", right_on = "Username")\
    .drop(['Full Name', 'Emp ID', 'Position Number', 'Banner Finance User ID'], axis = 1)

In [12]:
orgs_df.to_csv("output/fomprof-aduit-Orgs.csv", index = False)
funds_df.to_csv("output/fomprof-aduit-Funds.csv", index = False)